<div style="line-height:1.2;">

<h1 style="color:#f3de26; margin-bottom: 0.3em;">Multioutput classification 2</h1>

<h4 style="margin-top: 0.3em; margin-bottom: 1em;">Example of a sklearn MultiOutputClassifier with a RandomForestClassifier as base model.<br>Input data from Oimi Robot project for table5 games.</h4>

<div style="line-height:1.4; margin-bottom: 0.5em;">
    <h3 style="color: lightblue; display: inline; margin-right: 0.5em;">Keywords:</h3> 
    joblib.dump() + joblib.load() + ignore_warnings (ConvergenceWarning) + pandas apply() + LabelBinarizer + pandas.set_option() + pandas.option_context()
</div>

<div style="line-height:1.4;">
    <h3 style="color: red; display: inline; margin-right: 0.5em;">Notes:</h3>
    The dataset was not uploaded due to its magnitude. The same goes for the .sav trained model (5.1 Gb).
</div>


In [109]:
import joblib
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, LabelEncoder
from yellowbrick.model_selection import ValidationCurve
from sklearn.model_selection import train_test_split as tts

from sklearn.exceptions import ConvergenceWarning
try:
    from sklearn.utils._testing import ignore_warnings
except ImportError:
    from sklearn.utils.testing import ignore_warnings

from google.colab import drive

In [71]:
#%%script echo Uncomment if not on Colab
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%script echo Comment if not on Colab
my_dataset = pd.read_csv("./data_csv/reduced_dataset.csv", skiprows=None, engine='python')

In [72]:
my_dataset = pd.read_csv("/content/drive/My Drive/Colab_datasets/reduced_dataset.csv", skiprows=None, engine='python')

In [73]:
""" 
N.B.
no! take integers 
my_dataset['a2']= my_dataset['a2'].apply(str)
"""
pd.set_option('display.max_columns', None)
my_dataset[:3]

,lev,comple,s1,s2,s3,m1_diff,m2_diff,m3_diff,m4_diff,m5_diff,ord_q_games,a1,a2,n1,n2,m1_ord_quest,m1_ord_games_diff,m1_num_games,m2_ord_quest,m2_ord_games_diff,m2_num_games,m3_ord_quest,m3_ord_games_diff,m3_num_games,m4_ord_quest,m4_ord_games_diff,m4_num_games,m5_ord_quest,m5_ord_games_diff,m5_num_games
0,beginner,Easy,no,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,0,no,no,desc,desc,2,desc,casu,2,casu,casu,2,asc,casu,2,asc,asc,2
1,beginner,Easy,yes,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,0,no,no,same,casu,2,same,same,2,same,same,2,same,asc,2,same,same,2
2,beginner,Easy,no,yes,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,0,no,no,same,same,1,same,casu,1,same,same,1,casu,same,1,same,casu,1


In [74]:
# Get a list of all columns
cols = my_dataset.columns.tolist()

## Get the indices of the columns you want to switch
col1_index = cols.index('a2')
col2_index = cols.index('n2')

# Switch the columns in the list
cols[col2_index], cols[col1_index] = cols[col1_index], cols[col2_index]

# Re-index the DataFrame using the modified columns list
df = my_dataset[cols]

pd.set_option('display.max_columns', None)
df[:3]

,lev,comple,s1,s2,s3,m1_diff,m2_diff,m3_diff,m4_diff,m5_diff,ord_q_games,a1,n2,n1,a2,m1_ord_quest,m1_ord_games_diff,m1_num_games,m2_ord_quest,m2_ord_games_diff,m2_num_games,m3_ord_quest,m3_ord_games_diff,m3_num_games,m4_ord_quest,m4_ord_games_diff,m4_num_games,m5_ord_quest,m5_ord_games_diff,m5_num_games
0,beginner,Easy,no,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,no,0,desc,desc,2,desc,casu,2,casu,casu,2,asc,casu,2,asc,asc,2
1,beginner,Easy,yes,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,no,0,same,casu,2,same,same,2,same,same,2,same,asc,2,same,same,2
2,beginner,Easy,no,yes,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,no,0,same,same,1,same,casu,1,same,same,1,casu,same,1,same,casu,1


In [75]:
## Get the indices of the columns you want to switch
col1_index = cols.index('n2')
col2_index = cols.index('n1')

# Switch the columns in the list
cols[col2_index], cols[col1_index] = cols[col1_index], cols[col2_index]

# Re-index the DataFrame using the modified columns list
df_ok = df[cols]

pd.set_option('display.max_columns', None)
df_ok[:3]

,lev,comple,s1,s2,s3,m1_diff,m2_diff,m3_diff,m4_diff,m5_diff,ord_q_games,a1,n1,n2,a2,m1_ord_quest,m1_ord_games_diff,m1_num_games,m2_ord_quest,m2_ord_games_diff,m2_num_games,m3_ord_quest,m3_ord_games_diff,m3_num_games,m4_ord_quest,m4_ord_games_diff,m4_num_games,m5_ord_quest,m5_ord_games_diff,m5_num_games
0,beginner,Easy,no,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,no,0,desc,desc,2,desc,casu,2,casu,casu,2,asc,casu,2,asc,asc,2
1,beginner,Easy,yes,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,no,0,same,casu,2,same,same,2,same,same,2,same,asc,2,same,same,2
2,beginner,Easy,no,yes,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,no,0,same,same,1,same,casu,1,same,same,1,casu,same,1,same,casu,1


In [76]:
df_ok.isna().sum()

lev                  0
comple               0
s1                   0
s2                   0
s3                   0
m1_diff              0
m2_diff              0
m3_diff              0
m4_diff              0
m5_diff              0
ord_q_games          0
a1                   0
n1                   0
n2                   0
a2                   0
m1_ord_quest         0
m1_ord_games_diff    0
m1_num_games         0
m2_ord_quest         0
m2_ord_games_diff    0
m2_num_games         0
m3_ord_quest         0
m3_ord_games_diff    0
m3_num_games         0
m4_ord_quest         0
m4_ord_games_diff    0
m4_num_games         0
m5_ord_quest         0
m5_ord_games_diff    0
m5_num_games         0
dtype: int64

In [77]:
class training:
    #@ignore_warnings(category=ConvergenceWarning)
    def __init__(self, name, my_dataset):
        self.name = name
        self.my_dataset = my_dataset

        self.class_weights = {'m1_ord_quest':0.5,
                                'm1_ord_games_diff':0.5,
                                'm1_num_games':0.5,
                                'm2_ord_quest':0.5,
                                'm2_ord_games_diff':0.5,
                                'm2_num_games':0.5,
                                'm3_ord_quest':0.5,
                                'm3_ord_games_diff':0.5,
                                'm3_num_games':0.5,
                                'm4_ord_quest':0.5,
                                'm4_ord_games_diff':0.5,
                                'm4_num_games':0.5,
                                'm5_ord_quest':0.5,
                                'm5_ord_games_diff':0.5,
                                'm5_num_games':0.5}
        self.forest = RandomForestClassifier()                  #class_weight=self.class_weights)
        self.clf = MultiOutputClassifier(self.forest)

        ## Create a visualizer object
        self.viz = ValidationCurve(self.clf, param_name="max_depth", n_jobs=-1,
                                param_range=np.arange(1, 11), cv=10, scoring="accuracy")

    @ignore_warnings(category=ConvergenceWarning)
    def training_multioutput(self):
        # Shuffle
        self.my_dataset = shuffle(self.my_dataset, random_state=3)

        jobs_e0 = LabelBinarizer()
        jobs_e0.fit(self.my_dataset['lev'])
        jobs_e1 = LabelBinarizer()
        jobs_e1.fit(self.my_dataset['comple'])
        jobs_e2 = LabelBinarizer()
        jobs_e2.fit(self.my_dataset['s1'])
        jobs_e3 = LabelBinarizer()
        jobs_e3.fit(self.my_dataset['s2'])
        jobs_e4 = LabelBinarizer()
        jobs_e4.fit(self.my_dataset['s3'])
        jobs_e5 = LabelBinarizer()
        jobs_e5.fit(self.my_dataset['m1_diff'])
        jobs_e6 = LabelBinarizer()
        jobs_e6.fit(self.my_dataset['m2_diff'])
        jobs_e7 = LabelBinarizer()
        jobs_e7.fit(self.my_dataset['m3_diff'])
        jobs_e8 = LabelBinarizer()
        jobs_e8.fit(self.my_dataset['m4_diff'])
        jobs_e9 = LabelBinarizer()
        jobs_e9.fit(self.my_dataset['m5_diff'])
        jobs_e10 = LabelBinarizer()
        jobs_e10.fit(self.my_dataset['ord_q_games'])
        jobs_e11 = LabelBinarizer()
        jobs_e11.fit(self.my_dataset['a1'])
        jobs_e12 = LabelBinarizer()
        jobs_e12.fit(self.my_dataset['n1'])
        jobs_e13 = LabelBinarizer()
        jobs_e13.fit(self.my_dataset['n2'])

        trans0 = jobs_e0.transform(self.my_dataset['lev'])
        print(trans0)
        if trans0.shape[1] == 1:
            ohe0 = pd.DataFrame(trans0, columns=[f'lev_{jobs_e0.classes_[0]}'])
        else:
            ohe0 = pd.DataFrame(trans0, columns=[f'lev_{class_}' for class_ in jobs_e0.classes_])

        trans1 = jobs_e1.transform(self.my_dataset['comple'])
        if trans1.shape[1] == 1:
            ohe1 = pd.DataFrame(trans1, columns=[f'comple_{jobs_e1.classes_[0]}'])
        else:
            ohe1 = pd.DataFrame(trans1, columns=[f'comple_{class_}' for class_ in jobs_e1.classes_])

        trans2 = jobs_e2.transform(self.my_dataset['s1'])
        if trans2.shape[1] == 1:
            ohe2 = pd.DataFrame(trans2, columns=[f's1_{jobs_e2.classes_[0]}'])
        else:
            ohe2 = pd.DataFrame(trans2, columns=[f's1_{class_}' for class_ in jobs_e2.classes_])

        trans3 = jobs_e3.transform(self.my_dataset['s2'])
        if trans3.shape[1] == 1:
            ohe3 = pd.DataFrame(trans3, columns=[f's2_{jobs_e3.classes_[0]}'])
        else:
            ohe3 = pd.DataFrame(trans3, columns=[f's2_{class_}' for class_ in jobs_e3.classes_])

        trans4 = jobs_e4.transform(self.my_dataset['s3'])
        if trans4.shape[1] == 1:
            ohe4 = pd.DataFrame(trans4, columns=[f's3_{jobs_e4.classes_[0]}'])
        else:
            ohe4 = pd.DataFrame(trans4, columns=[f's3_{class_}' for class_ in jobs_e4.classes_])

        trans5 = jobs_e5.transform(self.my_dataset['m1_diff'])
        if trans5.shape[1] == 1:
            ohe5 = pd.DataFrame(trans5, columns=[f'm1_diff_{jobs_e5.classes_[0]}'])
        else:
            ohe5 = pd.DataFrame(trans5, columns=[f'm1_diff_{class_}' for class_ in jobs_e5.classes_])

        trans6 = jobs_e6.transform(self.my_dataset['m2_diff'])
        if trans6.shape[1] == 1:
            ohe6 = pd.DataFrame(trans6, columns=[f'm2_diff_{jobs_e6.classes_[0]}'])
        else:
            ohe6 = pd.DataFrame(trans6, columns=[f'm2_diff_{class_}' for class_ in jobs_e6.classes_])

        trans7 = jobs_e7.transform(self.my_dataset['m3_diff'])
        if trans7.shape[1] == 1:
            ohe7 = pd.DataFrame(trans7, columns=[f'm3_diff_{jobs_e7.classes_[0]}'])
        else:
            ohe7 = pd.DataFrame(trans7, columns=[f'm3_diff_{class_}' for class_ in jobs_e7.classes_])

        trans8 = jobs_e8.transform(self.my_dataset['m4_diff'])
        if trans8.shape[1] == 1:
            ohe8 = pd.DataFrame(trans8, columns=[f'm4_diff_{jobs_e8.classes_[0]}'])
        else:
            ohe8 = pd.DataFrame(trans8, columns=[f'm4_diff_{class_}' for class_ in jobs_e8.classes_])

        trans9 = jobs_e9.transform(self.my_dataset['m5_diff'])
        if trans9.shape[1] == 1:
            ohe9 = pd.DataFrame(trans9, columns=[f'm5_diff_{jobs_e9.classes_[0]}'])
        else:
            ohe9 = pd.DataFrame(trans9, columns=[f'm5_diff_{class_}' for class_ in jobs_e9.classes_])

        trans10 = jobs_e10.transform(self.my_dataset['ord_q_games'])
        if trans10.shape[1] == 1:
            ohe10 = pd.DataFrame(trans10, columns=[f'ord_q_games_{jobs_e10.classes_[0]}'])
        else:
            ohe10 = pd.DataFrame(trans10, columns=[f'ord_q_games_{class_}' for class_ in jobs_e10.classes_])

        trans11 = jobs_e11.transform(self.my_dataset['a1'])
        if trans11.shape[1] == 1:
            ohe11 = pd.DataFrame(trans11, columns=[f'a1_{jobs_e11.classes_[0]}'])
        else:
            ohe11 = pd.DataFrame(trans11, columns=[f'a1_{class_}' for class_ in jobs_e11.classes_])

        trans12 = jobs_e12.transform(self.my_dataset['n1'])
        if trans12.shape[1] == 1:
            ohe12 = pd.DataFrame(trans12, columns=[f'n1_{jobs_e12.classes_[0]}'])
        else:
            ohe12 = pd.DataFrame(trans12, columns=[f'n1_{class_}' for class_ in jobs_e12.classes_])

        trans13 = jobs_e13.transform(self.my_dataset['n2'])
        if trans13.shape[1] == 1:
            ohe13 = pd.DataFrame(trans13, columns=[f'n2_{jobs_e13.classes_[0]}'])
        else:
            ohe13 = pd.DataFrame(trans13, columns=[f'n2_{class_}' for class_ in jobs_e13.classes_])


        print("ohe11")
        print(self.my_dataset['a1'])
        print()
        print(jobs_e0)

        cippi = pd.DataFrame([self.my_dataset.pop(x) for x in ['lev',
                                                        'comple',
                                                        's1',
                                                        's2',
                                                        's3',
                                                        'm1_diff',
                                                        'm2_diff',
                                                        'm3_diff',
                                                        'm4_diff',
                                                        'm5_diff',
                                                        'ord_q_games',
                                                        'a1',
                                                        'n1',
                                                        'n2']]).T

        self.my_dataset = pd.concat([ohe0,ohe1,ohe2,ohe3,ohe4,ohe5,ohe6,ohe7,ohe8,ohe9,ohe10,ohe11,ohe12,ohe13,self.my_dataset], axis=1)

        columns_to_convert2 = [
                                'm1_ord_quest',
                                'm1_ord_games_diff',
                                'm1_num_games',
                                'm2_ord_quest',
                                'm2_ord_games_diff',
                                'm2_num_games',
                                'm3_ord_quest',
                                'm3_ord_games_diff',
                                'm3_num_games',
                                'm4_ord_quest',
                                'm4_ord_games_diff',
                                'm4_num_games',
                                'm5_ord_quest',
                                'm5_ord_games_diff',
                                'm5_num_games'
                            ]
        for column in columns_to_convert2:
            label_encoder = LabelEncoder()
            self.my_dataset[column] = label_encoder.fit_transform(self.my_dataset[column])

        Y = pd.DataFrame([self.my_dataset.pop(x) for x in ['m1_ord_quest',
                                                    'm1_ord_games_diff',
                                                    'm1_num_games',
                                                    'm2_ord_quest',
                                                    'm2_ord_games_diff',
                                                    'm2_num_games',
                                                    'm3_ord_quest',
                                                    'm3_ord_games_diff',
                                                    'm3_num_games',
                                                    'm4_ord_quest',
                                                    'm4_ord_games_diff',
                                                    'm4_num_games',
                                                    'm5_ord_quest',
                                                    'm5_ord_games_diff',
                                                    'm5_num_games']]).T

        print(self.my_dataset.head())
        print()
        print()
        print(self.my_dataset.head())
        print()
        print(Y.head())
        X_train, X_test, Y_train, Y_test = tts(self.my_dataset, Y, test_size=0.15)
        self.X_train_returned = X_train
        self.X_test_returned = X_test
        self.Y_te = Y_test
        self.Y_train_returned = Y_train

        w = np.asarray([2., 1.])
        self.clf.fit(X_train, Y_train)

        print(len(self.my_dataset.columns))
        print(len(Y.columns))
        print("ok,done")
        print()

        self.viz.fit(X_train, Y_train)
        self.viz.show()

        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(self.my_dataset)

        #filename = './final_OK_5_matches.sav'
        filename = '/content/final_OK_5_matches.sav'       #on Colab
        joblib.dump(self.clf, filename)

    def get_multiforest(self):
        return self.clf
    def get_x_train(self):
        return self.X_train_returned
    def get_x_test(self):
        return self.X_test_returned
    def get_y_train(self):
        return self.Y_train_returned
    def get_y_test(self):
        return self.Y_te

In [78]:
len(df_ok)

48332

<h3 style="color:#f3de26 "> Train: </h3>

In [79]:
on = training(name="train_test#1", my_dataset=df_ok)
on.training_multioutput()

x_train_for_aligment = on.get_x_train()
x_test_ret = on.get_x_test()
y_train_ret = on.get_y_train()
y_test_ret = on.get_y_test()

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
ohe11
22321    AAAA
27891    BCBB
41093    ABAA
40133    AAAA
2908     BBBA
         ... 
25544    ABAA
48056    BBBB
11513    BBBA
1688     ABAA
5994     CCBB
Name: a1, Length: 48332, dtype: object

LabelBinarizer()
   lev_beginner  comple_Easy  s1_no  s2_no  s3_no  m1_diff_Easy_1  \
0             0            0      0      1      1               0   
1             0            0      0      0      0               0   
2             0            0      0      0      0               0   
3             0            0      0      1      0               0   
4             0            0      0      0      0               0   

   m2_diff_Easy_1  m3_diff_Easy_1  m3_diff_Easy_2  m3_diff_Hard_1  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               1               0   
3               0               0               0       

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [80]:
y_test_ret[:3]

,m1_ord_quest,m1_ord_games_diff,m1_num_games,m2_ord_quest,m2_ord_games_diff,m2_num_games,m3_ord_quest,m3_ord_games_diff,m3_num_games,m4_ord_quest,m4_ord_games_diff,m4_num_games,m5_ord_quest,m5_ord_games_diff,m5_num_games
44465,1,2,1,3,1,2,3,1,2,2,3,1,0,0,0
7876,1,2,2,0,3,1,0,0,1,3,1,1,0,2,2
747,1,3,4,1,1,4,1,1,4,3,1,4,3,1,4


<h3 style="color:#f3de26 "> Predict: </h3>

In [81]:
d = {'lev': ['beginner'],
    'comple': ['Easy'],
    's1': ['no'],
    's2': ['no'],
    's3': ['no'],
    'm1_diff': ['Easy_1'],
    'm2_diff': ['Easy_1'],
    'm3_diff': ['Easy_1'],
    'm4_diff': ['Easy_1'],
    'm5_diff': ['Easy_1'],
    'ord_q_games': ['same'],
    'a1': ['AAAA'],
    'n1': ['no'],
    'n2': ['yes'],
    'a2': [0]
    }

arra1 = pd.DataFrame(data=d)

jobs_e0 = LabelBinarizer()
jobs_e0.fit(arra1['lev'])
jobs_e1 = LabelBinarizer()
jobs_e1.fit(arra1['comple'])
jobs_e2 = LabelBinarizer()
jobs_e2.fit(arra1['s1'])
jobs_e3 = LabelBinarizer()
jobs_e3.fit(arra1['s2'])
jobs_e4 = LabelBinarizer()
jobs_e4.fit(arra1['s3'])
jobs_e5 = LabelBinarizer()
jobs_e5.fit(arra1['m1_diff'])
jobs_e6 = LabelBinarizer()
jobs_e6.fit(arra1['m2_diff'])
jobs_e7 = LabelBinarizer()
jobs_e7.fit(arra1['m3_diff'])
jobs_e8 = LabelBinarizer()
jobs_e8.fit(arra1['m4_diff'])
jobs_e9 = LabelBinarizer()
jobs_e9.fit(arra1['m5_diff'])
jobs_e10 = LabelBinarizer()
jobs_e10.fit(arra1['ord_q_games'])
jobs_e11 = LabelBinarizer()
jobs_e11.fit(arra1['a1'])
jobs_e12 = LabelBinarizer()
jobs_e12.fit(arra1['n1'])
jobs_e13 = LabelBinarizer()
jobs_e13.fit(arra1['n2'])


trans0 = jobs_e0.transform(arra1['lev'])
print(trans0)
if trans0.shape[1] == 1:
    ohe0 = pd.DataFrame(trans0, columns=[f'lev_{jobs_e0.classes_[0]}'])
else:
    ohe0 = pd.DataFrame(trans0, columns=[f'lev_{class_}' for class_ in jobs_e0.classes_])

trans1 = jobs_e1.transform(arra1['comple'])
if trans1.shape[1] == 1:
    ohe1 = pd.DataFrame(trans1, columns=[f'comple_{jobs_e1.classes_[0]}'])
else:
    ohe1 = pd.DataFrame(trans1, columns=[f'comple_{class_}' for class_ in jobs_e1.classes_])

trans2 = jobs_e2.transform(arra1['s1'])
if trans2.shape[1] == 1:
    ohe2 = pd.DataFrame(trans2, columns=[f's1_{jobs_e2.classes_[0]}'])
else:
    ohe2 = pd.DataFrame(trans2, columns=[f's1_{class_}' for class_ in jobs_e2.classes_])

trans3 = jobs_e3.transform(arra1['s2'])
if trans3.shape[1] == 1:
    ohe3 = pd.DataFrame(trans3, columns=[f's2_{jobs_e3.classes_[0]}'])
else:
    ohe3 = pd.DataFrame(trans3, columns=[f's2_{class_}' for class_ in jobs_e3.classes_])

trans4 = jobs_e4.transform(arra1['s3'])
if trans4.shape[1] == 1:
    ohe4 = pd.DataFrame(trans4, columns=[f's3_{jobs_e4.classes_[0]}'])
else:
    ohe4 = pd.DataFrame(trans4, columns=[f's3_{class_}' for class_ in jobs_e4.classes_])

trans5 = jobs_e5.transform(arra1['m1_diff'])
if trans5.shape[1] == 1:
    ohe5 = pd.DataFrame(trans5, columns=[f'm1_diff_{jobs_e5.classes_[0]}'])
else:
    ohe5 = pd.DataFrame(trans5, columns=[f'm1_diff_{class_}' for class_ in jobs_e5.classes_])

trans6 = jobs_e6.transform(arra1['m2_diff'])
if trans6.shape[1] == 1:
    ohe6 = pd.DataFrame(trans6, columns=[f'm2_diff_{jobs_e6.classes_[0]}'])
else:
    ohe6 = pd.DataFrame(trans6, columns=[f'm2_diff_{class_}' for class_ in jobs_e6.classes_])

trans7 = jobs_e7.transform(arra1['m3_diff'])
if trans7.shape[1] == 1:
    ohe7 = pd.DataFrame(trans7, columns=[f'm3_diff_{jobs_e7.classes_[0]}'])
else:
    ohe7 = pd.DataFrame(trans7, columns=[f'm3_diff_{class_}' for class_ in jobs_e7.classes_])

trans8 = jobs_e8.transform(arra1['m4_diff'])
if trans8.shape[1] == 1:
    ohe8 = pd.DataFrame(trans8, columns=[f'm4_diff_{jobs_e8.classes_[0]}'])
else:
    ohe8 = pd.DataFrame(trans8, columns=[f'm4_diff_{class_}' for class_ in jobs_e8.classes_])

trans9 = jobs_e9.transform(arra1['m5_diff'])
if trans9.shape[1] == 1:
    ohe9 = pd.DataFrame(trans9, columns=[f'm5_diff_{jobs_e9.classes_[0]}'])
else:
    ohe9 = pd.DataFrame(trans9, columns=[f'm5_diff_{class_}' for class_ in jobs_e9.classes_])

trans10 = jobs_e10.transform(arra1['ord_q_games'])
if trans10.shape[1] == 1:
    ohe10 = pd.DataFrame(trans10, columns=[f'ord_q_games_{jobs_e10.classes_[0]}'])
else:
    ohe10 = pd.DataFrame(trans10, columns=[f'ord_q_games_{class_}' for class_ in jobs_e10.classes_])

trans11 = jobs_e11.transform(arra1['a1'])
if trans11.shape[1] == 1:
    ohe11 = pd.DataFrame(trans11, columns=[f'a1_{jobs_e11.classes_[0]}'])
else:
    ohe11 = pd.DataFrame(trans11, columns=[f'a1_{class_}' for class_ in jobs_e11.classes_])

trans12 = jobs_e12.transform(arra1['n1'])
if trans12.shape[1] == 1:
    ohe12 = pd.DataFrame(trans12, columns=[f'n1_{jobs_e12.classes_[0]}'])
else:
    ohe12 = pd.DataFrame(trans12, columns=[f'n1_{class_}' for class_ in jobs_e12.classes_])

trans13 = jobs_e13.transform(arra1['n2'])
if trans13.shape[1] == 1:
    ohe13 = pd.DataFrame(trans13, columns=[f'n2_{jobs_e13.classes_[0]}'])
else:
    ohe13 = pd.DataFrame(trans13, columns=[f'n2_{class_}' for class_ in jobs_e13.classes_])

[[0]]


In [82]:
arra1[:5]

,lev,comple,s1,s2,s3,m1_diff,m2_diff,m3_diff,m4_diff,m5_diff,ord_q_games,a1,n1,n2,a2
0,beginner,Easy,no,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,yes,0


In [83]:
arra1.isna().sum()

lev            0
comple         0
s1             0
s2             0
s3             0
m1_diff        0
m2_diff        0
m3_diff        0
m4_diff        0
m5_diff        0
ord_q_games    0
a1             0
n1             0
n2             0
a2             0
dtype: int64

In [84]:
arra1['a2']

0    0
Name: a2, dtype: int64

In [85]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(arra1[:3])

        lev comple  s1  s2  s3 m1_diff m2_diff m3_diff m4_diff m5_diff  \
0  beginner   Easy  no  no  no  Easy_1  Easy_1  Easy_1  Easy_1  Easy_1   

  ord_q_games    a1  n1   n2  a2  
0        same  AAAA  no  yes   0  


In [86]:
# Create dataframe from transposed array
cippi = pd.DataFrame([arra1.pop(x) for x in ['lev','comple','s1','s2','s3','m1_diff','m2_diff',
                                            'm3_diff','m4_diff','m5_diff','ord_q_games','a1','n1','n2']]).T
#cippi = arra.pop('lev')
#print(cippi)
cippi

,lev,comple,s1,s2,s3,m1_diff,m2_diff,m3_diff,m4_diff,m5_diff,ord_q_games,a1,n1,n2
0,beginner,Easy,no,no,no,Easy_1,Easy_1,Easy_1,Easy_1,Easy_1,same,AAAA,no,yes


In [87]:
arra1

,a2
0,0


In [88]:
arra1 = pd.concat([ohe0,
                ohe1,
                ohe2,
                ohe3,
                ohe4,
                ohe5,
                ohe6,
                ohe7,
                ohe8,
                ohe9,
                ohe10,
                ohe11,
                ohe12,
                ohe13,
                arra1], axis=1)

In [89]:
arra1

,lev_beginner,comple_Easy,s1_no,s2_no,s3_no,m1_diff_Easy_1,m2_diff_Easy_1,m3_diff_Easy_1,m4_diff_Easy_1,m5_diff_Easy_1,ord_q_games_same,a1_AAAA,n1_no,n2_yes,a2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
""" Check the presence of NaN values in each column. """
arra1.isna().sum()

lev_beginner        0
comple_Easy         0
s1_no               0
s2_no               0
s3_no               0
m1_diff_Easy_1      0
m2_diff_Easy_1      0
m3_diff_Easy_1      0
m4_diff_Easy_1      0
m5_diff_Easy_1      0
ord_q_games_same    0
a1_AAAA             0
n1_no               0
n2_yes              0
a2                  0
dtype: int64

In [91]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(arra1[:3])

   lev_beginner  comple_Easy  s1_no  s2_no  s3_no  m1_diff_Easy_1  \
0             0            0      0      0      0               0   

   m2_diff_Easy_1  m3_diff_Easy_1  m4_diff_Easy_1  m5_diff_Easy_1  \
0               0               0               0               0   

   ord_q_games_same  a1_AAAA  n1_no  n2_yes  a2  
0                 0        0      0       0   0  


In [92]:
arra1.isna().sum()

lev_beginner        0
comple_Easy         0
s1_no               0
s2_no               0
s3_no               0
m1_diff_Easy_1      0
m2_diff_Easy_1      0
m3_diff_Easy_1      0
m4_diff_Easy_1      0
m5_diff_Easy_1      0
ord_q_games_same    0
a1_AAAA             0
n1_no               0
n2_yes              0
a2                  0
dtype: int64

In [93]:
#with open('./data_csv/output_arra.csv', 'w') as f:
with open('/content/output_arra.csv', 'w') as f:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(arra1, file=f)

In [94]:
type(arra1)

pandas.core.frame.DataFrame

In [95]:
def align_columns(sample_df, reference_df):
    """ Align "sample_df" features with the one of received "reference_df".\\ 
    Add missing columns to the transformed sample and sets their values to 0. 
    """
    ### Add missing columns to sample and set them to 0
    for col in reference_df.columns:
        if col not in sample_df.columns:
            sample_df[col] = 0

    ### Drop extra columns from sample that aren't in reference
    for col in sample_df.columns:
        if col not in reference_df.columns:
            sample_df = sample_df.drop(col, axis=1)

    # Reorder columns to match the reference
    sample_df = sample_df[reference_df.columns]

    return sample_df

In [96]:
""" Align columns to solve the mismatch issue in the columns between the transformed single sample and the training data.
In fact, when encoding a single sample, the LabelBinarizer encoder isn't aware of the full scope of possible categories it might encounter, as it is only being fit on that single sample. 
"""
single_sample_aligned = align_columns(arra1, x_train_for_aligment)

In [97]:
single_sample_aligned.head()

,lev_beginner,comple_Easy,s1_no,s2_no,s3_no,m1_diff_Easy_1,m2_diff_Easy_1,m3_diff_Easy_1,m3_diff_Easy_2,m3_diff_Hard_1,m3_diff_Hard_2,m3_diff_Medium_1,m3_diff_Medium_2,m3_diff_Normal_1,m3_diff_Normal_2,m4_diff_Easy_1,m4_diff_Easy_2,m4_diff_Hard_1,m4_diff_Hard_2,m4_diff_Medium_1,m4_diff_Medium_2,m4_diff_Normal_1,m4_diff_Normal_2,m5_diff_Easy_1,ord_q_games_asc,ord_q_games_casu,ord_q_games_desc,ord_q_games_same,a1_AAAA,a1_ABAA,a1_BAAA,a1_BBAA,a1_BBBA,a1_BBBB,a1_BCBA,a1_BCBB,a1_CBAA,a1_CBBA,a1_CBCB,a1_CCBB,a1_CCBC,a1_CCCB,a1_CCCC,n1_no,n2_no,a2
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [98]:
# Get stored model trained previously
loaded_model = joblib.load("/content/final_OK_5_matches.sav")   #or filename = './final_OK_5_matches.sav' if not on Colab

#perceived_interaction = loaded_model.predict(arra1)
perceived_interaction = loaded_model.predict(single_sample_aligned)
print("perceived_interaction:")
print(perceived_interaction)

perceived_interaction:
[[1 3 1 1 3 2 3 1 1 3 3 0 3 1 0]]


In [99]:
%%script echo Skip, lead to InvalidIndexError: (slice(None, None, None), 0)
y_test_ret[:,0]

Skip, lead to InvalidIndexError: (slice(None, None, None), 0)


In [100]:
y_test_ret[:1]

,m1_ord_quest,m1_ord_games_diff,m1_num_games,m2_ord_quest,m2_ord_games_diff,m2_num_games,m3_ord_quest,m3_ord_games_diff,m3_num_games,m4_ord_quest,m4_ord_games_diff,m4_num_games,m5_ord_quest,m5_ord_games_diff,m5_num_games
44465,1,2,1,3,1,2,3,1,2,2,3,1,0,0,0


In [101]:
%%script echo Skipping since classification_report metrics is not supported for multiclass-multioutput.
report1 = classification_report(re[:1], perceived_interaction)
report1

Skipping since classification_report metrics is not supported for multiclass-multioutput.


In [105]:
y_pred = loaded_model.predict(x_test_ret)
y_true = y_test_ret

y_pred[:5], y_true[:5]

(array([[2, 1, 1, 1, 2, 2, 0, 2, 3, 3, 0, 3, 2, 3, 4],
        [1, 1, 3, 2, 3, 2, 3, 1, 2, 1, 1, 1, 1, 1, 4],
        [2, 1, 3, 1, 1, 2, 0, 2, 2, 1, 1, 1, 2, 1, 0],
        [1, 1, 2, 3, 3, 2, 1, 3, 2, 1, 3, 2, 3, 1, 2],
        [3, 3, 1, 3, 3, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1]]),
        m1_ord_quest  m1_ord_games_diff  m1_num_games  m2_ord_quest  \
 44465             1                  2             1             3   
 7876              1                  2             2             0   
 747               1                  3             4             1   
 35625             1                  1             2             2   
 6128              3                  3             1             1   
 
        m2_ord_games_diff  m2_num_games  m3_ord_quest  m3_ord_games_diff  \
 44465                  1             2             3                  1   
 7876                   3             1             0                  0   
 747                    1             4             1              

In [106]:
type(y_pred), type(y_true)

(numpy.ndarray, pandas.core.frame.DataFrame)

In [107]:
# Convert y_pred numpy array to a pandas DataFrame
y_pred = pd.DataFrame(y_pred, columns=y_true.columns)
print(y_pred.head())

   m1_ord_quest  m1_ord_games_diff  m1_num_games  m2_ord_quest  \
0             2                  1             1             1   
1             1                  1             3             2   
2             2                  1             3             1   
3             1                  1             2             3   
4             3                  3             1             3   

   m2_ord_games_diff  m2_num_games  m3_ord_quest  m3_ord_games_diff  \
0                  2             2             0                  2   
1                  3             2             3                  1   
2                  1             2             0                  2   
3                  3             2             1                  3   
4                  3             1             1                  1   

   m3_num_games  m4_ord_quest  m4_ord_games_diff  m4_num_games  m5_ord_quest  \
0             3             3                  0             3             2   
1             2 

<h3 style="color:#f3de26 "> Metrics: </h3>

In [118]:
## 1. Label-wise Metrics
labelwise_accuracy = [accuracy_score(y_true.iloc[:, i], y_pred.iloc[:, i]) for i in range(y_true.shape[1])]
avg_labelwise_accuracy = sum(labelwise_accuracy) / len(labelwise_accuracy)

## 2. Sample-wise Metrics
samplewise_accuracy = [accuracy_score(y_true.iloc[i, :], y_pred.iloc[i, :]) for i in range(y_true.shape[0])]
avg_samplewise_accuracy = sum(samplewise_accuracy) / len(samplewise_accuracy)

########### 3. Macro and Micro Averaging
## Reshape the matrices into long format
y_true_reshaped = y_true.values.reshape(-1)
y_pred_reshaped = y_pred.values.reshape(-1)

micro_precision = precision_score(y_true_reshaped, y_pred_reshaped, average='micro')
macro_precision = precision_score(y_true_reshaped, y_pred_reshaped, average='macro')
micro_recall = recall_score(y_true_reshaped, y_pred_reshaped, average='micro')
macro_recall = recall_score(y_true_reshaped, y_pred_reshaped, average='macro')

print("Avg Label-wise Accuracy:", avg_labelwise_accuracy)
print("Avg Sample-wise Accuracy:", avg_samplewise_accuracy)
print("Micro Precision:", micro_precision)
print("Macro Precision:", macro_precision)
print("Micro Recall:", micro_recall)
print("Macro Recall:", macro_recall)

Avg Label-wise Accuracy: 0.3299034482758621
Avg Sample-wise Accuracy: 0.3299034482758721
Micro Precision: 0.32990344827586204
Macro Precision: 0.2958983753410082
Micro Recall: 0.32990344827586204
Macro Recall: 0.284947066271948
